## Import libraries

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
from tensorflow.keras.applications.xception import  Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers.legacy import Adam #hadas
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image
from collections import defaultdict
from google.colab import drive
import pandas as pd
import os.path
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

##google drive

In [ ]:
PROJECT_PATH= '/content/drive/MyDrive/project/'

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Unzip data

In [ ]:
# DATA1='NSL-KDD'
DATA1='CICIDS2017'

In [ ]:
def unzip_folder(name_data,train_flag=True):
  if train_flag:
    data_zip_path=os.path.join(PROJECT_PATH,'data',name_data,'train_224_'+name_data )
    print(data_zip_path)
  else:
    data_zip_path=os.path.join(PROJECT_PATH,'data',name_data,'test_224_'+name_data )
  print(data_zip_path)
  !unzip -n {data_zip_path}



###Unzip train

In [ ]:
unzip_folder(DATA1)
# os.rename('train_224', 'train_224_'+DATA1)

Streaming output truncated to the last 5000 lines.
  inflating: train_224_CICIDS2017/6/4354.png  
  inflating: train_224_CICIDS2017/6/4355.png  
  inflating: train_224_CICIDS2017/6/4356.png  
  inflating: train_224_CICIDS2017/6/4358.png  
  inflating: train_224_CICIDS2017/6/4359.png  
  inflating: train_224_CICIDS2017/6/436.png  
  inflating: train_224_CICIDS2017/6/4360.png  
  inflating: train_224_CICIDS2017/6/4361.png  
  inflating: train_224_CICIDS2017/6/4362.png  
  inflating: train_224_CICIDS2017/6/4363.png  
  inflating: train_224_CICIDS2017/6/4365.png  
  inflating: train_224_CICIDS2017/6/4366.png  
  inflating: train_224_CICIDS2017/6/4367.png  
  inflating: train_224_CICIDS2017/6/4368.png  
  inflating: train_224_CICIDS2017/6/4370.png  
  inflating: train_224_CICIDS2017/6/4373.png  
  inflating: train_224_CICIDS2017/6/4375.png  
  inflating: train_224_CICIDS2017/6/4378.png  
  inflating: train_224_CICIDS2017/6/4379.png  
  inflating: train_224_CICIDS2017/6/438.png  
  inflating

###Unzip test

In [ ]:
unzip_folder(DATA1,train_flag=False)
# os.rename('test_224', 'test_224_'+DATA1)

Streaming output truncated to the last 5000 lines.
  inflating: test_224_CICIDS2017/6/23442.png  
  inflating: test_224_CICIDS2017/6/23455.png  
  inflating: test_224_CICIDS2017/6/2346.png  
  inflating: test_224_CICIDS2017/6/23468.png  
  inflating: test_224_CICIDS2017/6/23472.png  
  inflating: test_224_CICIDS2017/6/23480.png  
  inflating: test_224_CICIDS2017/6/23503.png  
  inflating: test_224_CICIDS2017/6/23507.png  
  inflating: test_224_CICIDS2017/6/23512.png  
  inflating: test_224_CICIDS2017/6/23516.png  
  inflating: test_224_CICIDS2017/6/2352.png  
  inflating: test_224_CICIDS2017/6/23522.png  
  inflating: test_224_CICIDS2017/6/23533.png  
  inflating: test_224_CICIDS2017/6/23557.png  
  inflating: test_224_CICIDS2017/6/23562.png  
  inflating: test_224_CICIDS2017/6/23567.png  
  inflating: test_224_CICIDS2017/6/23582.png  
  inflating: test_224_CICIDS2017/6/23586.png  
  inflating: test_224_CICIDS2017/6/23636.png  
  inflating: test_224_CICIDS2017/6/23639.png  
  inflating

##CUDA

In [ ]:
gpus=tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
if len(gpus):
  print(gpus)

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


##Constants

In [ ]:
#generate images from train set and validation set
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=128


## Generate Training and Test Images

###Generate Functions

In [ ]:
def make_name_folder(data):
  train_folder='./train_224_'+data+'/'
  test_folder='./test_224_'+data+'/'
  return train_folder,test_folder

def image_generator(data):
  train_folder,test_folder =make_name_folder(data)
  #Normalization
  train_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)
  train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=TARGET_SIZE,
        # color_mode=COLOR_MODE,
        # shuffle=False,
        class_mode='categorical',
        batch_size=BATCHSIZE,)
  test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=TARGET_SIZE,
        # color_mode=COLOR_MODE,
        batch_size=BATCHSIZE,
        shuffle=False,
        class_mode='categorical')
  return train_generator,test_generator

###Genrate NSL-KDD

In [ ]:
train_generator,test_generator=image_generator(DATA1)

Found 224000 images belonging to 7 classes.
Found 56000 images belonging to 7 classes.


# label

In [ ]:
def make_labels(generator):
  labels = []
  number_of_examples = len(generator.filenames)
  number_of_generator_calls = math.ceil(number_of_examples / (1.0 * BATCHSIZE))
  for _ in range(0,int(number_of_generator_calls)):
    img, label = generator.next()
    row,col=label.shape
    for i in range(int(row)):
      labels.append(np.where(label[i]==1.)[0][0])

  return labels


In [ ]:
train_label=make_labels(train_generator)

In [ ]:
len(train_label)

224000

In [ ]:
test_label=make_labels(test_generator)

In [ ]:
len(test_label)

56000

## Load 5 trained CNN models

In [ ]:

def make_path_file(data,dir,file):
  path1=os.path.join('training',data,dir,file)
  path=PROJECT_PATH+path1
  return path

def load_df(data):
  # path1=os.path.join('training',data,'best_model','best_model.csv')
  path=make_path_file(data,'best_model','best_model.csv')
  df=pd.read_csv(path)
  df=df.set_index('model')
  return df

def make_path_model(name_model,df,data):
  path=make_path_file(data,'best_model',df.loc[name_model ,'name file'])
  return path


def load_best_model(name_model,df,data):
  path=make_path_model(name_model,df,data)
  return load_model(path)

In [ ]:
best_model_df=load_df(DATA1)

In [ ]:
best_model_df

,name file
model,
Inception,inception_frozen:115_epochs=10_patience=2_lr=0...
InceptionResnet,inceptionresnet_frozen:503_epochs=20_patience=...
VGG16,VGG16_frozen:17_epochs=20_patience=3_lr=0.006_...
VGG19,VGG19_frozen:17_epochs=10_patience=4_lr=0.004_...
Xception,xception_frozen:133_epochs=15_patience=3_lr=0....


In [ ]:
path=make_path_model('Xception ',best_model_df,DATA1)
path

'/content/drive/MyDrive/project/training/CICIDS2017/best_model/xception_frozen:133_epochs=15_patience=3_lr=0.003_dropout=0.30000000000000004.h5'

In [ ]:
 #load model 1: xception
xception_model=load_best_model('Xception ',best_model_df,DATA1)

In [ ]:
 #load model 2: VGG16
vgg_model=load_best_model('VGG16',best_model_df,DATA1)

In [ ]:
 #load model 3: VGG19
vgg19_model=load_best_model('VGG19',best_model_df,DATA1)

In [ ]:
 #load model 4: inception
incep_model=load_best_model('Inception ',best_model_df,DATA1)

In [ ]:
 #load model 5: inceptionresnet
inres_model=load_best_model('InceptionResnet ',best_model_df,DATA1)

## Use the original CNN base models to make predictions

### function

In [ ]:
def make_df_predictions(models,generator,labels,verbose = 0):
  df_pred=pd.DataFrame()
  for model in models:
    y_pred = model.predict_generator(generator,verbose = verbose)
    df_model = pd.DataFrame(data=y_pred)
    df_pred=pd.concat([df_pred,df_model], axis=1)
  df_pred['label']=labels
  return df_pred




def calculate_performance(model, test_generator,test_labels,verbose = 0):
  test_generator.reset()
  y_pred = model.predict(test_generator,batch_size=BATCHSIZE,verbose = verbose)
  y_pred_bool = np.argmax(y_pred, axis=1)
  print(classification_report(test_labels, y_pred_bool))
  print('Accuracy score: ',accuracy_score(test_labels, y_pred_bool))



def performance_evaluation_model(model,test_generator,test_labels,verbose = 0):
  # df = pd.DataFrame(data=y_pred)
  test_generator.reset()
  results =model.evaluate(test_generator, batch_size=BATCHSIZE,verbose = verbose)
  print(f'evaluate: accuracy={results[1]}, loss= {results[0]}')
  calculate_performance(model,test_generator,test_labels,verbose = verbose)
  # print(classification_report(test_generator.labels, y_pred_bool))
  # return df

# def change_layer_name(model,name):
#   for i, layer in enumerate(model.layers):
#     layer._name  = layer.name+name


In [ ]:
performance_evaluation_model(xception_model,test_generator,test_label,verbose = 1)

438/438 [==============================] - 120s 252ms/step - loss: 0.0387 - accuracy: 0.9952
evaluate: accuracy=0.9951785802841187, loss= 0.03874097391963005
438/438 [==============================] - 110s 250ms/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8085
           1       1.00      0.99      1.00      7983
           2       1.00      1.00      1.00      7954
           3       0.98      1.00      0.99      7941
           4       1.00      1.00      1.00      7971
           5       1.00      1.00      1.00      8092
           6       0.99      1.00      1.00      7974

    accuracy                           1.00     56000
   macro avg       1.00      1.00      1.00     56000
weighted avg       1.00      1.00      1.00     56000

Accuracy score:  0.9951785714285715


In [ ]:
performance_evaluation_model(vgg_model,test_generator,test_label,verbose = 1)

438/438 [==============================] - 115s 255ms/step - loss: 0.0183 - accuracy: 0.9956
evaluate: accuracy=0.9955892562866211, loss= 0.01832381635904312
438/438 [==============================] - 110s 251ms/step
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      8085
           1       1.00      1.00      1.00      7983
           2       1.00      1.00      1.00      7954
           3       0.97      1.00      0.99      7941
           4       1.00      1.00      1.00      7971
           5       1.00      1.00      1.00      8092
           6       1.00      1.00      1.00      7974

    accuracy                           1.00     56000
   macro avg       1.00      1.00      1.00     56000
weighted avg       1.00      1.00      1.00     56000

Accuracy score:  0.9955892857142857


In [ ]:
performance_evaluation_model(vgg19_model,test_generator,test_label,verbose = 1)

438/438 [==============================] - 111s 252ms/step - loss: 0.0216 - accuracy: 0.9946
evaluate: accuracy=0.994607150554657, loss= 0.02156374230980873
438/438 [==============================] - 110s 251ms/step
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      8085
           1       1.00      1.00      1.00      7983
           2       1.00      1.00      1.00      7954
           3       0.97      0.99      0.98      7941
           4       1.00      1.00      1.00      7971
           5       1.00      1.00      1.00      8092
           6       1.00      1.00      1.00      7974

    accuracy                           0.99     56000
   macro avg       0.99      0.99      0.99     56000
weighted avg       0.99      0.99      0.99     56000

Accuracy score:  0.9946071428571429


In [ ]:
performance_evaluation_model(incep_model,test_generator,test_label,verbose = 1)

438/438 [==============================] - 115s 255ms/step - loss: 0.0229 - accuracy: 0.9959
evaluate: accuracy=0.9958928823471069, loss= 0.022908592596650124
438/438 [==============================] - 112s 253ms/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8085
           1       1.00      1.00      1.00      7983
           2       1.00      1.00      1.00      7954
           3       0.98      1.00      0.99      7941
           4       1.00      1.00      1.00      7971
           5       1.00      1.00      1.00      8092
           6       1.00      1.00      1.00      7974

    accuracy                           1.00     56000
   macro avg       1.00      1.00      1.00     56000
weighted avg       1.00      1.00      1.00     56000

Accuracy score:  0.9958928571428571


In [ ]:
performance_evaluation_model(inres_model,incep_model,test_label,verbose = 1)

438/438 [==============================] - 117s 256ms/step - loss: 0.0239 - accuracy: 0.9953
evaluate: accuracy=0.9953214526176453, loss= 0.023871349170804024
438/438 [==============================] - 114s 254ms/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      8085
           1       1.00      1.00      1.00      7983
           2       1.00      1.00      1.00      7954
           3       0.98      1.00      0.99      7941
           4       1.00      1.00      1.00      7971
           5       1.00      1.00      1.00      8092
           6       1.00      1.00      1.00      7974

    accuracy                           1.00     56000
   macro avg       1.00      1.00      1.00     56000
weighted avg       1.00      1.00      1.00     56000

Accuracy score:  0.9953214285714286


## make df

In [ ]:
test_pred=make_df_predictions([vgg_model,incep_model,inres_model],test_generator,test_label,verbose = 0)

In [ ]:
path=make_path_file(DATA1,'predictions','test_pred.csv')
test_pred.to_csv(path)

In [ ]:
train_pred=make_df_predictions([xception_model,incep_model,inres_model],train_generator,train_label,verbose = 0)

In [ ]:
path=make_path_file(DATA1,'predictions','train_pred.csv')
train_pred.to_csv(path)